## Importing Libraries 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as dt
import seaborn as sns
import re

C:\Users\pcz\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


We will be using :
- Regex (re) to extract and manipulate strings based on specific patterns.
- pandas for analysis.
- matlotlib and seaborn for visualization.
- emoji to deal with emojis.
- wordcloud for the most used words.

In [2]:
f=open('WhatsApp Chat with CSE 3C Official 2023-24.txt','r',encoding='utf-8')
data=f.read()

FileNotFoundError: [Errno 2] No such file or directory: 'WhatsApp Chat with CSE 3C Official 2023-24.txt'

In [ ]:
dummy=data.split('\n')
dummy

## Preprocessing

In [ ]:
pattern = r'\d{1,2}/\d{1,2}/\d{2,4}, \d{1,2}:\d{2}\s[APap][Mm]\s-\s'
messages = re.split(pattern, data)[1:]
print(len(messages))

In [ ]:
dates=re.findall(pattern,data)
print(len(dates))

In [ ]:
messages

In [ ]:
string= "10/16/23, 10:11\u202fPM - "
string = string.split(',')
date, time = string[0], string[1]
time=time.split('-')
time=time[0].strip()
print(date+" and "+time)

In [ ]:
def gettimeanddate(string):
     string = string.split(',')
     date, time = string[0], string[1]
     time=time.split('-')
     time=time[0].strip()

     return date+" "+time

In [ ]:
df = pd.DataFrame({'user_message':messages, 'message_date': dates})
df['message_date'] = df['message_date'].apply(lambda text: gettimeanddate(text))
df.rename(columns={'message_date': 'date'}, inplace=True)
df.head()

In [ ]:
df['user_message']

In [ ]:
users = []
messages = []
for message in df['user_message']:
    entry = re.split('([\w\W]+?):\s', message)
    if entry[1:]:
        users.append(entry[1])
        messages.append(entry[2])
    else:
        users.append('whatsapp notification')
        messages.append(entry[0])

df['user'] = users
df['message'] = messages

def getstring(text):
    return text.split('\n')[0]

df['message']=df['message'].apply(lambda text:getstring(text))

df.drop(['user_message'],axis=1)
df=df[['message','date','user']]

df=df.rename(columns={'message':'Message','date':'Date','user':'User'})
df.head()

In [ ]:
df.shape

In [ ]:
df['Only date'] = pd.to_datetime(df['Date']).dt.date
df['Year'] = pd.to_datetime(df['Date']).dt.year
df['Month_num'] = pd.to_datetime(df['Date']).dt.month
df['Month'] = pd.to_datetime(df['Date']).dt.month_name()
df['Day'] = pd.to_datetime(df['Date']).dt.day
df['Day_name'] = pd.to_datetime(df['Date']).dt.day_name()
df['Hour'] = pd.to_datetime(df['Date']).dt.hour
df['Minute'] = pd.to_datetime(df['Date']).dt.minute

df.head()

In [ ]:
df.columns

In [ ]:
df[df['User']=='Seema Mam']

In [ ]:
words = []
for message in df['Message']:
    words.extend(message.split())
    
print(len(words))

In [ ]:
df[df['Message']=='<Media omitted>']

In [ ]:
df[df['User']=='whatsapp notification']

## Functions

In [ ]:
def fetch_stats(selected_user, df):

    if (selected_user != "Overall"):
        df = df[df['user'] == selected_user]

    # fetch the number of msgs
    num_messages = df.shape[0]

    # fetch the number of words
    words = []
    for message in df['Message']:
        words.extend(message.split())

    # fetch the number of media msgs
    num_media_msgs = df[df['Message'] == '<Media omitted>']

    # fetch the number of linls shared
    links = []
    for message in df['Message']:
        links.extend(extract.find_urls(message))

    return num_messages,len(words), num_media_msgs.shape[0], len(links)


In [ ]:
def most_active_users(df):
    x = df['User'].value_counts().head()
    df = round((df['User'].value_counts() / df.shape[0]) * 100, 2).reset_index().rename(columns={'index': 'name', 'User': 'percent'})
    return x, df


In [ ]:
def create_wordcloud(selected_user, df):

    f = open('stop_hinglish.txt', 'r')
    stop_words = f.read()

    if (selected_user != "Overall"):
        df = df[df['User'] == selected_user]

    temp = df[df['User'] != 'whatsapp notification']
    temp = temp[temp['Message'] != '<Media omitted>\n']
    temp = temp[temp['Message'] != 'This message was deleted\n']

    words = []

    for message in temp['Message']:
        for word in message.lower().split():
            if word not in stop_words:
                words.append(word)

    words = pd.DataFrame(words, columns=['words'])
    wc = WordCloud(width=500, height=500, min_font_size=10, background_color='white')
    df_wc = wc.generate(words['words'].str.cat(sep=" "))
    return df_wc

In [ ]:
def most_common_words(selected_user, df):

    f = open('stop_hinglish.txt', 'r')
    stop_words = f.read()

    if (selected_user != "Overall"):
        df = df[df['User'] == selected_user]

    temp = df[df['User'] != 'whatsapp notification']
    temp = temp[temp['Message'] != '<Media omitted>\n']
    temp = temp[temp['Message'] != 'This message was deleted\n']

    words = []

    for message in temp['Message']:
        for word in message.lower().split():
            if word not in stop_words:
                words.append(word)

    pure_words = []
    for message in words:
        pure_words.extend([c for c in message if c in emoji.demojize(message)])

    most_common_df = pd.DataFrame(Counter(words).most_common(20))
    return most_common_df

In [ ]:
def emoji_stats(selected_user, df):
    if (selected_user != "Overall"):
        df = df[df['User'] == selected_user]

    emojis = []
    for message in df['Message']:
        emojis.extend([c for c in message if c not in emoji.demojize(message)])

    emoji_df = pd.DataFrame(Counter(emojis).most_common((len(Counter(emojis)))))
    return emoji_df

In [ ]:
def monthly_timeline(selected_user, df):
    if (selected_user != "Overall"):
        df = df[df['User'] == selected_user]

    timeline = df.groupby(['Year', 'Month_num', 'Month']).count()['Message'].reset_index()

    time = []
    for i in range(timeline.shape[0]):
        time.append(timeline['Month'][i] + "-" + str(timeline['Year'][i]))

    timeline['Time'] = time

    return timeline

def daily_timeline(selected_user, df):
    if (selected_user != "Overall"):
        df = df[df['User'] == selected_user]

    daily_timeline = df.groupby('Only_date').count()['Message'].reset_index()

    return daily_timeline

def week_activity_map(selected_user, df):
    if (selected_user != "Overall"):
        df = df[df['User'] == selected_user]

    return df['Day_name'].value_counts()

def month_activity_map(selected_user, df):
    if (selected_user != "Overall"):
        df = df[df['User'] == selected_user]

    return df['Month'].value_counts()

def activity_heatmap(selected_user, df):
    if (selected_user != "Overall"):
        df = df[df['User'] == selected_user]

    user_heatmap = df.pivot_table(index='Day_name', values='Message', aggfunc='count').fillna(0)

    return user_heatmap

### Top 10 most active users

In [ ]:
df2 = df.copy()    
df2 = df2[df2.User != "group_notification"]
top10df = df2.groupby("User")["Message"].count().sort_values(ascending=False)

# Final Data Frame
top10df = top10df.head(10).reset_index()
top10df

In [ ]:
# For better readablity;
import matplotlib
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (9, 5)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

In [ ]:
top10df['initials'] = ''
for i in range(10):
    top10df.initials[i] = top10df.User[i].split()[0][0] + top10df.User[i].split()[1][0]
    
top10df.initials[7] = "Me"    # That's me
top10df.initials[8] = "DT"

# Visualisation

### Plotting a simple line graph.

In [ ]:
sns.set_style("whitegrid")

plt.figure(figsize=(12, 6))

plt.plot(top10df.initials, top10df.Message, 'o--c')  

plt.xlabel('Users')
plt.ylabel('Total number of messages')

plt.title("Number of messages sent by group members.")
plt.legend(['Messages'])

### Plotting a bar chart and line graph together.

In [ ]:
sns.set_style("whitegrid")

plt.figure(figsize=(12, 6))

plt.bar(top10df.initials, top10df.Message) 
plt.plot(top10df.initials, top10df.Message, 'o--c');  